# ***Rating & Visit_mode***

In [ ]:
!pip install streamlit
!pip install pyngrok

In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

col1, col2 = st.columns([0.7,0.7])

with col1:
    st.image("/content/drive/MyDrive/mini_project_4_db/m4_pickle_files/Tourism template.jpg",width=600)
with col2:
    st.image("/content/drive/MyDrive/mini_project_4_db/m4_pickle_files/images_temp.jpg",width=600)

st.title('🤖Smart Travel Recommender✨')
data=pd.read_csv('/content/drive/MyDrive/mini_project_4_db/m4_pickle_files/data.csv')
fil_data = data[['Continent','Region','Country','Attraction','VisitMonth','VisitMode','CityName','Rating']]
main_menu=fil_data.groupby(['Continent','Region','Country','Attraction','VisitMonth','VisitMode','CityName'])['Rating'].mean().reset_index()
main_menu.drop('Region',axis=1,inplace=True)

st.write("Discover your next dream destination! Our smart system recommends the best tourist spots based on your travel style, location, and ratings.")
st.subheader("How It Works")
st.markdown("""
1. 🎯 Select your preferences (Country, City, Mode of Visit).
2. 🌍 View top-rated tourist places.
3. 🧳 Save your favorite destinations and start planning!
""")
st.subheader("Key Features")
st.markdown("""
- 🚀 Real-time destination suggestions.
- 🏖️ Personalized based on your choices.
- 📊 Data-driven recommendations.
- 🌟 Explore attractions by ratings, region, and season.
""")

st.sidebar.header("Recommender🌍")
options=st.sidebar.selectbox('',['Select_Type','Rating','Visit_mode'])

if options == 'Rating':
  st.header('Predict Rating')
  month = data['VisitMonth'].unique()
  VisitMonth = st.selectbox('VisitMonth',[ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])
  CityName = st.selectbox('CityName',data['CityName'].unique())
  Attraction = st.selectbox('Attraction',data['Attraction'].unique())
  VisitMode = st.selectbox('VisitMode',data['VisitMode'].unique())
  submitted = st.button('Predict')
  if submitted:
    features={'VisitMonth': VisitMonth, 'CityName' : CityName , 'Attraction' : Attraction, 'VisitMode' : VisitMode}

    pre_freatures=pd.DataFrame([features])
    with open('/content/drive/MyDrive/mini_project_4_db/m4_pickle_files/label_encoder.pkl','rb') as enco:
        encoder=pickle.load(enco)
    for col in pre_freatures.columns:
        if col in encoder:
            pre_freatures[col] = encoder[col].transform(pre_freatures[col])
    with open('/content/drive/MyDrive/mini_project_4_db/m4_pickle_files/RFC_Ratings.pkl','rb') as model:
        rfc_model=pickle.load(model)
        prediction=rfc_model.predict(pre_freatures)
        st.header('Approximate Ratings')
        st.subheader(f"{prediction[0]}")

if options == 'Visit_mode':
  st.header('Predict Visit_mode')
  vm_month = data['VisitMonth'].unique()
  vm_year = data['VisitYear'].unique()
  vm_year.sort()
  vm_VisitYear = st.selectbox('VisitYear','2023')
  vm_VisitMonth = st.selectbox('VisitMonth',[ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])
  vm_Rating = st.selectbox('Rating',data['Rating'].unique())
  vm_CityName = st.selectbox('CityName',data['CityName'].unique())
  vm_Attraction = st.selectbox('Attraction',data['Attraction'].unique())
  vm_AttractionType = st.selectbox('AttractionType',data['AttractionType'].unique())
  vm_Country = st.selectbox('Country',data['Country'].unique())
  vm_submitted = st.button('Predict')
  if vm_submitted:
    vm_features={'VisitYear': vm_VisitYear,'VisitMonth': vm_VisitMonth,'Rating' : vm_Rating ,'Attraction' : vm_Attraction,
               'AttractionType' : vm_AttractionType,'CityName' : vm_CityName ,   'Country' : vm_Country}

    vm_pre_freatures=pd.DataFrame([vm_features])
    with open('/content/drive/MyDrive/mini_project_4_db/m4_pickle_files/label_encoder.pkl','rb') as enco:
        encoder=pickle.load(enco)
    for col in vm_pre_freatures.columns:
        if col in encoder:
            vm_pre_freatures[col] = encoder[col].transform(vm_pre_freatures[col])
    with open('/content/drive/MyDrive/mini_project_4_db/m4_pickle_files/KNNClassifier_VM.pkl','rb') as model:
        knn_model=pickle.load(model)
        vm_prediction=knn_model.predict(vm_pre_freatures)
        st.header('Suggesting Visitmode')
        sv=['Business', 'Couples', 'Family', 'Friends', 'Solo']
        st.subheader(f"{sv[vm_prediction[0]]}")


In [ ]:
from pyngrok import ngrok
ngrok.kill()

In [ ]:
!npm install localtunnel

In [ ]:
!streamlit run /content/app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

# ***Recommendation***

In [ ]:
#Installing packages

!pip install scikit-surprise
# Uninstall current NumPy completely
!pip uninstall -y numpy
# Reinstall specific version
!pip install numpy==1.23.5 --force-reinstall
!pip install streamlit
!pip install pyngrok

In [ ]:
from surprise import Dataset, Reader, SVD
from surprise.model_selection import cross_validate
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
import pickle
from surprise import Dataset, Reader, SVD
from surprise.model_selection import cross_validate

col1, col2 = st.columns([0.7,0.7])

with col1:
    st.image("/content/drive/MyDrive/mini_project_4_db/m4_pickle_files/Tourism template.jpg",width=600)
with col2:
    st.image("/content/drive/MyDrive/mini_project_4_db/m4_pickle_files/images_temp.jpg",width=600)

st.title('🤖Smart Travel Recommender✨')
data=pd.read_csv('/content/drive/MyDrive/mini_project_4_db/m4_pickle_files/data.csv')
fil_data = data[['Continent','Region','Country','Attraction','VisitMonth','VisitMode','CityName','Rating']]
main_menu=fil_data.groupby(['Continent','Region','Country','Attraction','VisitMonth','VisitMode','CityName'])['Rating'].mean().reset_index()
main_menu.drop('Region',axis=1,inplace=True)

st.write("Discover your next dream destination! Our smart system recommends the best tourist spots based on your travel style, location, and ratings.")
st.subheader("How It Works")
st.markdown("""
1. 🎯 Select your preferences (Country, City, Mode of Visit).
2. 🌍 View top-rated tourist places.
3. 🧳 Save your favorite destinations and start planning!
""")
st.subheader("Key Features")
st.markdown("""
- 🚀 Real-time destination suggestions.
- 🏖️ Personalized based on your choices.
- 📊 Data-driven recommendations.
- 🌟 Explore attractions by ratings, region, and season.
""")

st.sidebar.header("Recommender🌍")
options=st.sidebar.selectbox('',['Select_Type','Collaborative Filtering','Content-Based'])

if options=='Collaborative Filtering':
  st.header('Content-Based Recommendation')
  #user related Recommendation
  coll_rec = data[['UserId','CityId','Rating','CityName','Country']]

  UserId = st.selectbox('UserId',data['UserId'].unique())

# Load into Surprise format
  reader = Reader(rating_scale=(1, 5))
  data_surprise = Dataset.load_from_df(coll_rec[['UserId','CityId','Rating']], reader)
  train_data=data_surprise.build_full_trainset()

  with open('/content/drive/MyDrive/mini_project_4_db/m4_pickle_files/SVD.pkl','rb') as model:
    svd_model=pickle.load(model)


  def recommend_svd(user_id, n=10):
    all_cities = coll_rec['CityId'].unique()
    rated = coll_rec[coll_rec['UserId'] == user_id]['CityId'].tolist()
    unrated = [city for city in all_cities if city not in rated]

    preds = [svd_model.predict(user_id, cid) for cid in unrated]
    preds.sort(key=lambda x: x.est, reverse=True)

    top_cities = [pred.iid for pred in preds[:n]]
    return coll_rec[coll_rec['CityId'].isin(top_cities)][['CityName', 'Country']].drop_duplicates()
  predict_coll=st.button('Predict')
  if predict_coll:
    coll_output = recommend_svd(UserId)
    st.header("Recommended Places")
    st.write(coll_output)

if options=='Content-Based':
    st.header('Content-Based Recommendation')
    content_rec=data[['CityName','VisitMode','AttractionType','Attraction','Country']].value_counts().reset_index()
    content_rec['rec_data'] =  content_rec['AttractionType'] + ' ' + content_rec['Attraction'] + ' ' + content_rec['Country']
    from sklearn.feature_extraction.text import TfidfVectorizer

    tfid = TfidfVectorizer()

    x_rec = tfid.fit_transform(content_rec['rec_data'] )
    from sklearn.metrics.pairwise import cosine_similarity

    co_sim = cosine_similarity(x_rec,x_rec)
    def recommend_place(AttractionType):
        idx = content_rec[content_rec['AttractionType']==AttractionType].index
        idx=idx[0]
        sim = list(enumerate(co_sim[idx]))
        sim = sorted(sim,key=lambda x:x[1],reverse=True)[:50]
        att=[i[0] for i in sim]
        return content_rec.iloc[att][['CityName','VisitMode','Country']]
    att=data['AttractionType'].unique()
    att_input=st.selectbox('SelectAttractionType',att)
    predict_content = st.button('Predict')
    if predict_content:
      att_output=recommend_place(att_input)
      st.header("Recommended Places")
      st.write(att_output)

st.sidebar.header("Tourist Place Directory")

continent_list = main_menu['Continent'].unique()
country_list = main_menu['Country'].unique()
country_list.sort()
attraction_list = main_menu['Attraction'].unique()
visitmonth_list = main_menu['VisitMonth'].unique()
visitmonth_list.sort()
visitmode_list = main_menu['VisitMode'].unique()


country = st.sidebar.selectbox("Country",country_list)
visitmonth = st.sidebar.selectbox("VisitMonth",visitmonth_list)
visitmode = st.sidebar.selectbox("VisitMode",visitmode_list)

show = st.sidebar.button('Show Places')
if show:
    filtered_data = main_menu[(main_menu['Country']==country) & (main_menu['VisitMonth']==visitmonth) & (main_menu['VisitMode']==visitmode)]
    st.header("Recommended Places")
    st.write(filtered_data)


In [ ]:
from pyngrok import ngrok
ngrok.kill()

In [ ]:
!npm install localtunnel

In [ ]:
!streamlit run /content/app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com